In [3]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display # to display images
import matplotlib.pyplot as plt
import os
import sys
import glob
import cv2
sys.path.append('/workspace/yo/google_od_api/src/lib/')
from utility import custom_decorator

DISPLAY = 'SAVE' # SHOW & SAVE
USAGE = 'PREDICTS' # PREDICTS & LABELS
THRESHOLD = 0.7
IMAGE_PATH = '/workspace/datasets/BDD100k/VOC_test/JPEGImages/'
DETECTIONS_PATH = '/workspace/yo/google_od_api/predict_results/momentum_1280_720/'
SAVE_PATH = '/workspace/yo/google_od_api/predict_results/momentum_1280_720/pictures/'
SAVE_FILENAME_EXTENSION = '.jpg'
BOUNDINGBOX_WIDTH = 3
LIMIT_NUM = 100
CLASS_COLOR = 'default'
IMAGE_SIZE = (18, 12)

ColorTable = dict({'RED': (255, 0, 0),
                  'ORANGE': (255, 165, 0),
                  'YELLOW': (255, 255, 0),
                  'GREEN': (0, 255, 0),
                  'BLUE': (0, 127, 255),
                  'INDIGO': (0, 0, 255),
                  'PURPLE': (139, 0, 255),
                  'WHITE': (255, 255, 255),
                  'BLACK': (0, 0, 0)}
)
ClassColor = dict(
        default = {'bike': ColorTable['RED'],
                   'bus': ColorTable['ORANGE'],
                   'car': ColorTable['YELLOW'],
                   'motor': ColorTable['GREEN'],
                   'person': ColorTable['BLUE'],
                   'rider': ColorTable['INDIGO'],
                   'truck': ColorTable['PURPLE'],
                 }
)

def drawTextWithMask(draw, loc, text, text_color, mask_color, font):
    ascent, descent = font.getmetrics()
    (width, baseline), (offset_x, offset_y) = font.font.getsize(text)
    left, top, right, bottom = font.getmask(text).getbbox()
    mask_loc = (loc[0], loc[1]+2*bottom, loc[0]+right, loc[1])
    draw.rectangle(mask_loc, mask_color)
    draw.text(loc, text, text_color, font=font)

def drawBoundingBox(draw, file, font):
    if os.path.exists(file):
        with open(file, 'r') as predict_fd:
            for line in predict_fd:
                line = line.rstrip().split(' ')
                if USAGE=='LABELS':
                    text_info = line[0]
                    bb_str = line[1:5]
                    bb = [int(i) for i in bb_str]
                    bb = tuple(bb)
                    draw.rectangle(bb, outline  = ClassColor[CLASS_COLOR][text_info], width=BOUNDINGBOX_WIDTH)
                    drawTextWithMask(draw, bb[0:2], ''.join((text_info)), ClassColor[CLASS_COLOR][text_info],
                                     ColorTable['BLACK'], font=font)
                else:
                    text_info = line[0:2]
                    bb_str = line[2:6]
                    bb = [int(i) for i in bb_str]
                    bb = tuple(bb)
                    if float(text_info[1]) > THRESHOLD:
                        draw.rectangle(bb, outline  =  ClassColor[CLASS_COLOR][text_info[0]], width=BOUNDINGBOX_WIDTH)
                        drawTextWithMask(draw, bb[0:2], ''.join((text_info[0])), ClassColor[CLASS_COLOR][text_info[0]],
                                         ColorTable['BLACK'], font=font)

def readFileTotalLines(file_path):
    count = 0
    thefile = open(file_path, 'r')
    while 1:
        buffer = thefile.read(8192*1024)
        if not buffer: break
        count += buffer.count('\n')
    thefile.close(  )
    return count

@custom_decorator.timeit
def saveImg(img, filename, index, save_path):
    img.save(''.join((save_path, filename, SAVE_FILENAME_EXTENSION)))
    print("index: ", index+1)
    
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    else:
        pass
    
def usageSwitch():
    if USAGE=='PREDICTS':
        boundingbox_path = ''.join((DETECTIONS_PATH, 'detections/'))
        save_path = ''.join((SAVE_PATH, 'threshoid_', str(THRESHOLD), '/'))
    elif USAGE=='LABELS':
        boundingbox_path = ''.join((DETECTIONS_PATH, 'groundtruths/'))
        save_path = ''.join((SAVE_PATH, 'groundtruths/'))
    else:
        print('ERROR COMMAND!!')
        boundingbox_path = None
        save_path = None
    return boundingbox_path, save_path

def main():
    font = ImageFont.truetype("/usr/share/fonts/truetype/lato/Lato-Black.ttf", size=28, encoding="unic")
    bounding_box_path, save_path = usageSwitch()
    mkdir(save_path)
    for index, image_path in enumerate(glob.glob(bounding_box_path+'*.txt')):
        filename = image_path.rstrip().split('/')[-1].split('.')[0]
        print(filename)
        img_name = ''.join((IMAGE_PATH,filename,'.jpg'))
        img = Image.open(img_name)
        width, height = img.size
        draw = ImageDraw.Draw(img)
        drawBoundingBox(draw, ''.join((bounding_box_path, filename,'.txt')), font)
        if DISPLAY == 'SHOW':
            display(img)
        elif DISPLAY == 'SAVE':
            saveImg(img, filename, index, save_path)
        else:
            print('ERROR COMMAND!!')
            break
        if index+1 >= LIMIT_NUM:
            break
    print("success")
    
main()

b23bb45f-ddeea1d8
index:  1
saveImg function spend 0.057734012603759766 sec
b7f452e4-1e8b6f3a
index:  2
saveImg function spend 0.05608773231506348 sec
b82e22f1-40b5aa11
index:  3
saveImg function spend 0.05184221267700195 sec
c5b7b466-eac86195
index:  4
saveImg function spend 0.05107545852661133 sec
bb43e180-96b21056
index:  5
saveImg function spend 0.04626822471618652 sec
b44a7e5f-1db66cfe
index:  6
saveImg function spend 0.04300236701965332 sec
b953efc6-f40d4290
index:  7
saveImg function spend 0.04082083702087402 sec
c4c62e31-64421eb6
index:  8
saveImg function spend 0.04687857627868652 sec
b3a9281a-e8ba5d66
index:  9
saveImg function spend 0.03822827339172363 sec
c5d79098-0b67a544
index:  10
saveImg function spend 0.04051971435546875 sec
c7cc0645-95e6a224
index:  11
saveImg function spend 0.03646659851074219 sec
bc140c47-28089dad
index:  12
saveImg function spend 0.04228377342224121 sec
c66c630d-8cc5464a
index:  13
saveImg function spend 0.04647016525268555 sec
c2186a76-6757a4be
in